# 量子过程直接保真度估计

*版权所有 (c) 2023 百度量子计算研究所，保留所有权利。*

## 内容概要

量子过程层析技术能够给我们提供量子过程的完整信息，但是它十分消耗资源，并且大部分情况我们想要的信息只是实际制备量子过程的保真度。量子过程直接保真度估计（Direct Fidelity Estimation of Quantum Process，DFE）是一种高效获得量子过程保真度的方法。

本教程将介绍面向量子过程的直接保真度估计的理论及其在[百度量子平台](https://quantum.baidu.com/)上的实践。

## 理论


### 保真度估计

理想酉信道 $\mathcal{U}$ 和实际量子过程 $\mathcal{N}$ 之间的纠缠保真度如下式

$$
    \mathcal{F}(\mathcal{U}, \mathcal{N}) = \frac{1}{d^2}\textrm{Tr}[\mathcal{M}_\mathcal{U}^\dagger \mathcal{M}_\mathcal{N}],
$$

其中 $\mathcal{M}_\mathcal{U}$ 和 $\mathcal{M}_\mathcal{N}$ 是 $\mathcal{U}$ 和$\mathcal{N}$的矩阵表示。 我们将 $P_k(k=1,\cdots,d^2)$ 定义为所有可能的 Pauli 算符，其中 $d=2^n$，$n$ 是量子比特的数量。 我们知道 $\mathcal{U}$ 和 $\mathcal{N}$ 都可以在 Pauli 基下展开，即

$$
\begin{align}
    \textrm{Tr}[\mathcal{M}_\mathcal{U}^\dagger \mathcal{M}_\mathcal{N}]
    &= \frac{1}{d^2} \sum_{k, k'} \langle \langle P_k \vert \mathcal{U} \vert P_{k'} \rangle \rangle \langle \langle P_k \vert \mathcal{N} \vert P_{k'} \rangle \rangle \\
    &= \sum_{k,k'} \chi_{\mathcal{U}}(k,k') \chi_{\mathcal{N}}(k,k'),
\end{align}
$$

其中 $\chi_{\mathcal{U}}(k,k')$ 的定义为

$$
\begin{align}
    \chi_{\mathcal{U}}(k,k')
    &= \langle \langle P_k \vert \mathcal{U} \vert P_{k'} \rangle \rangle \\
    &= \textrm{Tr}[P_k \mathcal{U}(P_k')],
\end{align}
$$

$\chi_{\mathcal{N}(k,k')}$ 的定义方式类似。然后，我们构建纠缠保真度的估计器，如下所示：

$$
\begin{align}
    \mathcal{F}(\mathcal{U}, \mathcal{N})
    &= \frac{1}{d^2} \sum_{k,k'} \chi_{\mathcal{U}}(k,k') \chi_{\mathcal{N}}(k,k') \\
    &= \sum_{k,k'}  \frac{\chi_{\mathcal{N}}(k,k')}{\chi_{\mathcal{U}}(k,k')} \frac{\chi_{\mathcal{U}}(k,k')^2}{d^2} \\
    &= \sum_{k,k'} X(k,k') \textrm{Pr}(k,k').
\end{align}
$$

接下来，我们描述实验过程。

首先，我们根据概率分布

$$
\begin{align}
    \textrm{Pr}(k,k')
    &= \frac{1}{d^2} \chi_{\mathcal{U}}(k,k')^2,
\end{align}
$$

对 Pauli 算符对进行 $l$ 次采样，采样结果是 $((P_{k_1},P_{k'_1}), \cdots, (P_{k_l}, P_{k_l'}))$。 然后对于每对 Pauli 算符 $(P_{k_i}, P_{k_i'})$，有

$$
\begin{align}
    X(k_i, k_i')
    &= \frac{\chi_{\mathcal{N}}(k_i, k_i')}{\chi_{\mathcal{U}}(k_i, k_i')}
    &= \frac{\textrm{Tr}[P_{k_i} \mathcal{N}(P_{k_i'})]}{\textrm{Tr}[P_{k_i} \mathcal{U}(P_{k_i'})]}.
\end{align}
$$

我们按照均匀分布随机选择 $P_{k_i'}$ 的特征值，表示为 $\lambda_{a_{ij}} \in \left\{ -1,1 \right\}$，其中 $a_{ij} \in \{1,2, \cdots, d\}$；然后我们准备相应的本征态 $\vert \psi_{a_{ij}} \rangle \langle \psi_{a_{ij}} \vert$ 并将 $\mathcal{N}$ 作用于该本征态。 接下来，我们测量 Pauli 可观测量 $P_{k_i}$，并得到结果 $A_{ij} \in \{-1,1\}$。 我们定义

$$
\begin{align}
B_{ij}
&= \lambda_{a_{ij}} A_{ij}.
\end{align}
$$

于是有

$$
\begin{align}
    \mathbb{E}[B_{ij}]
    &= \mathbb{E}_{\lambda_{a_{ij}}}[\mathbb{E}[ \lambda_{a_{ij}} A_{ij} \vert \lambda_{a_{ij}}]] \\
    &= \mathbb{E}_{\lambda_{a_{ij}}}[\mathbb{E}[A_{ij}] \lambda_{a_{ij}} ] \\
    &= \sum_{a_{ij}=1}^d \frac{1}{d} \lambda_{a_{ij}} \mathbb{E}[A_{ij}] \\
    &= \textrm{Tr}(P_{k_i} \mathcal{N}(P_{k_i'})),
\end{align}
$$

等于 $\chi_{\mathcal{N}}(k_i, k_i')$。 所以得到

$$
\begin{align}
 \tilde{X}(k_i, k_i')
 &= \frac{\sum_{j=1}^{m_i} B_{ij}}{m_i \chi_{\mathcal{U}}(k_i, k_i')}，
\end{align}
$$

其中，$m_i$ 表示第 $i$ 个 Pauli 需要的测量操作数量。

最后，我们的估计器是

$$
\begin{align}
    \tilde{\mathcal{F}}(\mathcal{U}, \mathcal{N})
    &= \frac{1}{l} \sum_{i=1}^l \tilde{X}(k_i, k_i').
\end{align}
$$

### 参数设置

还有两个问题需要我们去处理：

1. 我们需要采样多少个 Pauli 算符对？
2. 对于每个 Pauli 测量，我们需要重复多少次测量操作？

下面我们将解决这两个问题。首先，我们定义两个变量，给定误差 $\epsilon$ 和错误率 $\delta$ 。利用 Chebyshev's 不等式和 Hoeffding's 不等式，我们便可以计算得到采样次数 $l$ 和每次采样对应的测量次数 $m_i$ [1]。

我们可以做如下总结，真实保真度 $\mathcal{F}(\rho, \sigma) \in [\tilde{\mathcal{F}}(\mathcal{U}, \mathcal{N})-2\epsilon, \tilde{\mathcal{F}}(\mathcal{U}, \mathcal{N})+2\epsilon]$ 的概率 $\geq 1-2\delta$ ，其中 $\tilde{\mathcal{F}}(\mathcal{U}, \mathcal{N})$ 是保真度的估计值。


## 实践

下面，我们将演示如何将直接保真度估计应用到贝尔态的构建电路上。

首先，我们导入一些必要的包。

In [ ]:
import QCompute
import qcompute_qep.estimation as estimation

然后，我们在第二个和第三个量子比特上，搭建制备贝尔态的量子电路。

In [ ]:
qp = QCompute.QEnv()  # qp is short for "quantum program", instance of QProgram
qp.Q.createList(3)
QCompute.H(qp.Q[1])
QCompute.CX(qp.Q[1], qp.Q[2])

![DFE](./figures/dfe-bell-state.png "图1: 贝尔态制备电路。 ")

现在，我们成功搭建如上电路。

接下来我们需要选择量子计算机的类型，可以选择模拟器或量子计算机真机，这里我们选择本地模拟器。剩下的步骤就很简单了，只需要调用我们的 DFEProcess 模块则可完成直接保真度估计。这里，我们设置参数 $\epsilon=\delta=0.05$ ，并且将目标比特设置为第二个和第三个量子比特（`qubits=[1, 2]`）。

In [ ]:
# 使用本地模拟器
qc = QCompute.BackendName.LocalBaiduSim2

# 请登录“量易伏”平台（https://quantum-hub.baidu.com/）获取 Token
# QCompute.Define.hubToken = "Token"
# qc = QCompute.BackendName.CloudBaiduQPUQian

# Initialize a DFEProcess instance
dfe = estimation.DFEProcess()

# Call the estimate procedure
qubits = [1, 2]
dfe.estimate(qp, qc, qubits=qubits, epsilon=0.05, delta=0.05)

最后，我们可以计算得到制备贝尔态的量子电路的保真度。

In [ ]:
print('Fidelity of the Bell gate on qubits {} is: F = {:.5f}'.format(qubits, dfe.fidelity))

## References

[1] Flammia, Steven T., and Yi-Kai Liu. "Direct fidelity estimation from few Pauli measurements." **[Physical Review Letters](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.106.230501)** 106.23 (2011): 230501.

[2] da Silva, Marcus P., Olivier Landon-Cardinal, and David Poulin. "Practical characterization of quantum devices without tomography." **[Physical Review Letters](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.107.210404)** 107.21 (2011): 210404.

[3] Steffen, Lars, et al. "Experimental Monte Carlo quantum process certification." **[Physical Review Letters](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.108.260506)** 108.26 (2012): 260506.

[4] Kliesch, Martin, and Ingo Roth. "Theory of Quantum System Certification." **[PRX Quantum](https://journals.aps.org/prxquantum/abstract/10.1103/PRXQuantum.2.010201)** 2.1 (2021): 010201.